In [187]:
import pandas as pd
from skimpy import skim 

In [188]:
sh2021 = pd.read_csv('school_holiday_2021_22.csv')
sh2022 = pd.read_csv('school_holiday_2022_23.csv')
sh2023 = pd.read_csv('school_holiday_2023_24.csv')
sh2024 = pd.read_csv('school_holiday_2024_25.csv')
lookup_table = pd.read_csv('../time_to_settlement_period_lookup_table.csv')
lookup_table['Date'] = lookup_table['settlementDate']
lookup_table = lookup_table[['Date', 'settlementPeriod', 'startTime']]
lookup_table['Date'] = lookup_table['Date'].astype('str')

In [189]:
lookup_table

,Date,settlementPeriod,startTime
0,2020-01-01,1,2020-01-01T00:00:00Z
1,2020-01-01,2,2020-01-01T00:30:00Z
2,2020-01-01,3,2020-01-01T01:00:00Z
3,2020-01-01,4,2020-01-01T01:30:00Z
4,2020-01-01,5,2020-01-01T02:00:00Z
...,...,...,...
63140,2024-12-31,40,2024-12-31T19:30:00Z
63141,2024-12-31,45,2024-12-31T22:00:00Z
63142,2024-12-31,46,2024-12-31T22:30:00Z
63143,2024-12-31,47,2024-12-31T23:00:00Z


In [190]:
# skim(sh2021)
# skim(lookup_table)
# lookup_table

In [191]:
def clean_data(data):
    temp_df = data[data['Local_Authority'] == 'Total'].reset_index(drop = True)
    temp_df['Date'] = temp_df['Date'].astype('str')
    temp_df = temp_df[['Date', 'School_Holiday']]
    result = temp_df.merge(lookup_table, on = 'Date' , how='left')
    result = result.sort_values(by = 'startTime')
    return result

In [192]:
sh2021_clean = clean_data(sh2021)
sh2022_clean = clean_data(sh2022)
sh2023_clean = clean_data(sh2023)

dfs = [sh2021_clean, sh2022_clean, sh2023_clean]
output_df = pd.concat(dfs).reset_index(drop = True)

In [193]:
sh2024_clean = sh2024
sh2024_clean = sh2024_clean[sh2024_clean['Local_Authority'] == 'Total'].reset_index(drop = True)
sh2024_clean = sh2024_clean[['Date', 'School_Holiday']]
sh2024_clean['Date'] = pd.to_datetime(sh2024_clean['Date'], format='%d/%m/%Y')
lookup_table['Date'] = pd.to_datetime(lookup_table['Date'], format='%Y-%m-%d')
result = sh2024_clean.merge(lookup_table, on = 'Date' , how='inner')

combined_output = pd.concat([result, output_df]).drop_duplicates()
# combined_output
result



,Date,School_Holiday,settlementPeriod,startTime
0,2024-06-27,0.028,1,2024-06-26T23:00:00Z
1,2024-06-27,0.028,2,2024-06-26T23:30:00Z
2,2024-06-27,0.028,3,2024-06-27T00:00:00Z
3,2024-06-27,0.028,4,2024-06-27T00:30:00Z
4,2024-06-27,0.028,5,2024-06-27T01:00:00Z
...,...,...,...,...
6161,2024-12-31,1.000,40,2024-12-31T19:30:00Z
6162,2024-12-31,1.000,45,2024-12-31T22:00:00Z
6163,2024-12-31,1.000,46,2024-12-31T22:30:00Z
6164,2024-12-31,1.000,47,2024-12-31T23:00:00Z


In [194]:
combined_output = combined_output[['Date', 'settlementPeriod', 'School_Holiday']]
combined_output['Date'] = pd.to_datetime(combined_output['Date'], format='mixed')
combined_output

,Date,settlementPeriod,School_Holiday
0,2024-06-27,1.0,0.028
1,2024-06-27,2.0,0.028
2,2024-06-27,3.0,0.028
3,2024-06-27,4.0,0.028
4,2024-06-27,5.0,0.028
...,...,...,...
34893,2024-06-09,40.0,0.011
34894,2024-06-09,45.0,0.011
34895,2024-06-09,46.0,0.011
34896,2024-06-09,47.0,0.011


In [197]:
combined_output = combined_output.sort_values(['Date', 'settlementPeriod']).reset_index()
combined_output['Date'] = combined_output['Date'].astype('str')
combined_output

,index,Date,settlementPeriod,School_Holiday
0,0,2021-06-19,1.0,0.002
1,1,2021-06-19,2.0,0.002
2,2,2021-06-19,3.0,0.002
3,3,2021-06-19,4.0,0.002
4,4,2021-06-19,5.0,0.002
...,...,...,...,...
40921,6161,2024-12-31,40.0,1.000
40922,6162,2024-12-31,45.0,1.000
40923,6163,2024-12-31,46.0,1.000
40924,6164,2024-12-31,47.0,1.000


In [198]:
combined_output.to_csv('school_holiday_processed.csv')